# 任务6 OpenCV人脸识别算法基于视频流的方式实现人脸检测

## 职业能力目标

- 能够使用线程的方式完成实时人脸检测。

## 任务描述

本实验将实现用opencv自带的人脸库对USB摄像头实时采集并显示在触摸屏的画面进行人脸检测并标注人脸框。

## 任务要求

- 使用线程实现人脸实时检测。

## 任务实施

## 1. 回顾图像人脸检测

详细代码解释这里不再赘述，如有遗忘，可以查看[任务5.图像人脸检测](./任务5.图像人脸检测.ipynb)重新回顾

In [ ]:
import cv2

#设置图片路径
img_path = './img/face1.jpg'
#载入带有人脸的图片
img = cv2.imread(img_path)

#将彩色图片转换为灰度图
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

#载入人脸检测的Cascade模型
FaceCascade = cv2.CascadeClassifier('./haar/haarcascade_frontalface_default.xml')

#检测画面中的人脸
faces = FaceCascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5)

#遍历返回的face数组
for face in faces:
    #解析tuple类型的face位置数据
    #(x, y): 左上角坐标值
    #w: 人脸矩形区域的宽度
    #h: 人脸矩形区域的高度
    (x, y, w, h) = face
    #在原彩图上绘制矩形
    cv2.rectangle(img, (x, y), (x+w, y+h), (0, 255, 0), 4)
    
#创建一个窗口 名字叫做Face
cv2.namedWindow('Face',flags=cv2.WINDOW_NORMAL | cv2.WINDOW_KEEPRATIO | cv2.WINDOW_GUI_EXPANDED)
cv2.setWindowProperty('Face', cv2.WND_PROP_FULLSCREEN, cv2.WINDOW_FULLSCREEN) # 全屏展示

#在窗口Face上面展示图片img
cv2.imshow('Face', img)
#等待任意按键按下
cv2.waitKey(5000)

## 2. 视频流人脸检测
通过将[任务5.图像人脸检测](./任务5.图像人脸检测.ipynb)的代码结合循环，循环对每一帧图像进行人脸检测，就能实现视频流实时人脸检测。

OpenCV读取usb摄像头的视频流，需要用到`VideoCapture`类。如果你在操作过程中，摄像头读取失败， 我们还为你提供了问题[排查手册](./doc/摄像头排查手册.txt)。

### 2.1 导入相应的模块

- `cv2`：实现图像处理和计算机视觉方面的很多通用算法。
- `threading`：threading模块提供了管理多个线程执行的API。

In [ ]:
import threading
import cv2 # 引入opencv库函数

### 2.2 视频流人脸检测类

线程编写和视频流显示实验参考[项目1：使用OpenCV实现人脸检测中的2_opencv实现视频流的调用](../../项目1：使用OpenCV实现人脸检测/2_opencv实现视频流的调用/实验2opencv实现视频流的调用.ipynb)，这里不再赘述

1. 通过上一份实验**3.1 视频流的图像显示与退出**实验改写线程视频流类，来实现人脸检测功能
2. 在`init`函数中传入定义`标志位变量`、`打开摄像头`、`载入人脸检测的Cascade模型`
3. 将`灰度图转换`、`人脸检测`、`绘制矩形框`结合循环，通过循环的方式反复的将摄像头读取到的每一帧进行这3步操作，就能够进行视频流人脸检测。
3. 在`run`函数中`构建视频窗口`
    - 循环体里`读取摄像头图像`、`灰度图转换`、`人脸检测`、`绘制矩形框`、`更新显示图片`、`图像显示的时长`
4. 在`stop`函数中定义`标志位变量`、`摄像头释放`、`窗口释放`
5. 用线程的方式运行函数，再对视频进行人脸检测


In [ ]:
class FaceDetectionThread(threading.Thread):
    def __init__(self):
        super(FaceDetectionThread, self).__init__()
        self.working = True  # 循环标志位
        self.cap = cv2.VideoCapture(0)  # 打开摄像头      
        self.FaceCascade = cv2.CascadeClassifier('./haar/haarcascade_frontalface_default.xml')#载入人脸检测的Cascade模型        
    def run(self):
        # 构建视频的窗口
        cv2.namedWindow('Face',flags=cv2.WINDOW_NORMAL | cv2.WINDOW_KEEPRATIO | cv2.WINDOW_GUI_EXPANDED)
        cv2.setWindowProperty('Face', cv2.WND_PROP_FULLSCREEN, cv2.WINDOW_FULLSCREEN) # 全屏展示
        while self.working:
            ret, frame = self.cap.read()# 读取摄像头图像
            if ret:
                gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)#将彩色图片转换为灰度图 
                faces = self.FaceCascade.detectMultiScale(gray,scaleFactor=1.1,minNeighbors=5)# 检测图像中的人脸并返回绘制矩形框的值
                for face in faces:
                    (x, y, w, h) = face
                    cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 4)#在原彩图上绘制矩形
                cv2.imshow('Face',frame)# 更新窗口“Face”中的图片
                cv2.waitKey(1)# 等待按键事件发生 等待1ms
    def stop(self):
        self.working = False
        self.cap.release()# 释放VideoCapture
        cv2.destroyAllWindows()# 销毁所有的窗口
        print("退出线程")

实例化一个`FaceDetectionThread()`线程类，实例化对象为`a`

线程对象`a`调用`start()`方法, 开始执行`FaceDetectionThread()`线程类中的`run()`函数。

In [ ]:
a = FaceDetectionThread()
a.start()

实例化对象`a`调用线程类中的`stop()`函数，来退出线程。

In [ ]:
a.stop()

### <font color=red>动手实验</font>

仿照上述代码，自行完成视频流人脸检测代码

按照以下要求完成实验：
1. 在`<1>`处，定义一个线程类命名为`FaceTest`
2. 在`__init__`函数中：
    - 在`<2>`处，定义循环标志位`self.working`赋值为`True`
    - 在`<3>`处，使用`cv2.VideoCapture(0)`打开摄像头赋值给`self.cap`对象
    - 在`<4>`处，使用`cv2.CascadeClassifier`载入`haarcascade_frontalface_default.xml`人脸检测模型赋值给`self.FaceCascade`
    
3. 在`run`函数中：

  - 在`<5>`处，使用`cv2.namedWindow`创建显示窗口命名为`FaceTest`,属性设置为`可调整大小`，`保持图像比例` 
  
    循环中：
    - 在`<6>`处，使用`self.cap.read()`读取图像将返回值赋值给`ret`和`frame`
    - 在`<7>`处，使用`cv2.cvtColor`将`frame`中的彩色图像通过`cv2.COLOR_BGR2GRAY`转换成灰度图，并且返回给变量`gray`
    - 在`<8>`处，使用`self.FaceCascade.detectMultiScale`得到矩形框返回值`faces`，参数定义：输入图像为`gray`、图像缩小的比例为`1.1`、匹配成功所需要的周围矩形框的数目为`4`
    - 在`<9>`处，通过循环遍历`faces`，使用`cv2.rectangle`对图像进行矩形框绘制，颜色为`(255, 0, 0)`，矩形框线粗为`3`
    - 在`<10>`处，使用`cv2.imshow`在窗口`FaceTest`中显示图像`frame`，设置`cv2.waitKey()`为1毫秒

4. 在`stop`函数中：
    - 在`<11>`处，将`self.working`赋值为`False`退出循环
    - 在`<12>`处，使用`self.cap.release()`和`cv2.destroyAllWindows()`来释放`VideoCapture`和销毁所有的窗口

**能够成功开启线程并在显示屏进行视频流人脸检测，退出线程有`退出线程`输出，则表示实验完成。**

In [ ]:
import threading
import cv2 # 引入opencv库函数

class FaceTest(threading.Thread):
    def __init__(self):
        super(FaceTest, self).__init__()
        self.working = True  # 循环标志位
        self.cap = cv2.VideoCapture(0)  # 打开摄像头
        if not self.cap.isOpened():
            print("Cannot open camera")           
        #载入人脸检测的Cascade模型
        self.FaceCascade = cv2.CascadeClassifier('./haar/haarcascade_frontalface_default.xml')
        
    def run(self):
        # 构建视频的窗口
        cv2.namedWindow('FaceTest',flags=cv2.WINDOW_NORMAL | cv2.WINDOW_KEEPRATIO)
        cv2.setWindowProperty('FaceTest', cv2.WND_PROP_FULLSCREEN, cv2.WINDOW_FULLSCREEN) # 全屏展示
        while self.working:
            # 读取摄像头图像
            ret, frame = self.cap.read()
            if ret:
                #将彩色图片转换为灰度图
                gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
                # 检测图像中的人脸并返回绘制矩形框的值
                faces = self.FaceCascade.detectMultiScale(
                    gray,
                    scaleFactor=1.1,
                    minNeighbors=5
                )

                #遍历返回的face数组
                for face in faces:
                    #解析tuple类型的face位置数据
                    #(x, y): 左上角坐标值
                    #w: 人脸矩形区域的宽度
                    #h: 人脸矩形区域的高度
                    (x, y, w, h) = face
                    #在原彩图上绘制矩形
                    cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 3)

                # 更新窗口“image_win”中的图片
                cv2.imshow('FaceTest', frame)
                # 等待按键事件发生 等待1ms
                cv2.waitKey(1)

    def stop(self):
        self.working = False
        # 释放VideoCapture
        self.cap.release()
        # 销毁所有的窗口
        cv2.destroyAllWindows()
        print("退出线程")

<details>
<summary><font color=red size=3>点击查看答案</font></summary>
<pre><code>

```python
import threading
import cv2 # 引入opencv库函数

class FaceTest(threading.Thread):
    def __init__(self):
        super(FaceTest, self).__init__()
        self.working = True  # 循环标志位
        self.cap = cv2.VideoCapture(0)  # 打开摄像头
        if not self.cap.isOpened():
            print("Cannot open camera")           
        #载入人脸检测的Cascade模型
        self.FaceCascade = cv2.CascadeClassifier('./haar/haarcascade_frontalface_default.xml')
        
    def run(self):
        # 构建视频的窗口
        cv2.namedWindow('FaceTest',flags=cv2.WINDOW_NORMAL | cv2.WINDOW_KEEPRATIO)
        cv2.setWindowProperty('FaceTest', cv2.WND_PROP_FULLSCREEN, cv2.WINDOW_FULLSCREEN) # 全屏展示
        while self.working:
            # 读取摄像头图像
            ret, frame = self.cap.read()
            if ret:
                #将彩色图片转换为灰度图
                gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
                # 检测图像中的人脸并返回绘制矩形框的值
                faces = self.FaceCascade.detectMultiScale(
                    gray,
                    scaleFactor=1.1,
                    minNeighbors=5
                )

                #遍历返回的face数组
                for face in faces:
                    #解析tuple类型的face位置数据
                    #(x, y): 左上角坐标值
                    #w: 人脸矩形区域的宽度
                    #h: 人脸矩形区域的高度
                    (x, y, w, h) = face
                    #在原彩图上绘制矩形
                    cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 3)

                # 更新窗口“image_win”中的图片
                cv2.imshow('FaceTest', frame)
                # 等待按键事件发生 等待1ms
                cv2.waitKey(1)

    def stop(self):
        self.working = False
        # 释放VideoCapture
        self.cap.release()
        # 销毁所有的窗口
        cv2.destroyAllWindows()
        print("退出线程")
```

</code></pre>
</details>

实例化一个`FaceTest()`线程类，实例化对象为`a`

线程对象`a`调用`start()`方法, 开始执行`FaceTest()`线程类中的`run()`函数。

In [ ]:
a = FaceTest()
a.start()

实例化对象`a`调用线程类中的`stop()`函数，来退出线程。

注意：请在本实验结束时，务必退出线程，否则可能由于该线程占用摄像头导致其它实验无法正常进行。

In [ ]:
a.stop()

## 任务小结

本次实验的收获：

- 学会了使用线程实现人脸实时检测